This notebook exhibits a way in which data for investigating the increase in runtime with system number can be generated.

In [1]:
import sys
sys.path.insert(0,'..')
import oqupy
import numpy as np
from oqupy import contractions
import time
import csv

sigma_z = oqupy.operators.sigma("z")
sigma_plus = oqupy.operators.sigma("+")
sigma_minus = oqupy.operators.sigma("-")

alpha = 0.25
nu_c = 227.9
T = 39.3
omega_0 = 0.0
omega_c = -30.4
Omega = 303.9

kappa = 15.2
Gamma_down = 30.4
Gamma_up = 0.8 * Gamma_down

gammas = [ lambda t: Gamma_down, lambda t: Gamma_up]
lindblad_operators = [ lambda t: sigma_minus, lambda t: sigma_plus]

def H_MF(t, a):
    return 0.5 * omega_0 * sigma_z +\
        0.5 * Omega * (a * sigma_plus + np.conj(a) * sigma_minus)
        
def single_field_eom(t, state, a):
    expect_val = np.matmul(sigma_minus, state).trace()
    return -(1j * omega_c + kappa) * a - 0.5j * Omega * expect_val

correlations = oqupy.PowerLawSD(alpha=alpha,
                                zeta=1,
                                cutoff=nu_c,
                                cutoff_type='gaussian',
                                temperature=T)

bath = oqupy.Bath(0.5 * sigma_z, correlations)
initial_field = np.sqrt(0.05)               # Note n_0 = <a^dagger a>(0) = 0.05
initial_state = np.array([[0,0],[0,1]])     # spin down

tempo_parameters = oqupy.TempoParameters(dt=3.2e-3, dkmax=20, epsrel=10**(-5))
start_time = 0.0
end_time = 0.5

process_tensor = oqupy.pt_tempo_compute(bath=bath,
                                        start_time=start_time,
                                        end_time=end_time,
                                        parameters=tempo_parameters)

--> PT-TEMPO computation:
100.0%  156 of  156 [########################################] 00:00:03
Elapsed time: 4.0s


In [2]:
def create_csv(filename, num_of_systems, time_taken, overwrite = True):
    """ This function creates a CSV file with 2 columns: one for the numebr of quantum systems,
    and the other for the time taken to compute the dynamics of the super-system.

    Arguments:
    filename -- name of CSV file
    num_of_systems -- number of quantum systems in a super-system
    time_taken -- time taken to compute dynamics of super-system
    overwrite -- boolean specifying whether the csv should overwrite a pre-existing file (default True)
    """
    write_mode = "wt"       # default mode of writing to csv (will cause overwriting of pre-existing file)
    
    if overwrite == False:
        write_mode = "at"   # this mode adds text to the end of a pre-existing file
        
    with open(filename, write_mode, newline = "", encoding = "utf-8") as runtime_data:
        csv_writer = csv.DictWriter(runtime_data, ["Number of Systems", "Runtime"])

        # add headers to columns of the CSV once a file is created or overwritten
        if overwrite == True:
            csv_writer.writeheader() 

        csv_writer.writerow({"Number of Systems":num_of_systems, "Runtime":time_taken})
    if overwrite == True:
        print("CSV", filename, "created successfully")
    else:
        print("CSV", filename, "updated successfully")

def compute_n_systems(n, filename, overwrite = False):
    """Return runtime for the dynamics of a super-system with n systems. 
    """
    if n == 1:
        overwrite = True

    fractions = [1/n for i in range(n)]
    def field_eom(t, states, field):
        sx_exp_list = [np.matmul(sigma_minus, state).trace() for state in states]
        sx_exp_weighted_sum = sum([fraction*sx_exp for fraction, sx_exp in zip(fractions, sx_exp_list)])
        return -(1j*omega_c+kappa)*field - 0.5j*Omega*sx_exp_weighted_sum
    initial_state_list = [initial_state for i in range(n)]
    
    system = oqupy.TimeDependentSystemWithField(
        H_MF,
        single_field_eom,
        gammas=gammas,
        lindblad_operators=lindblad_operators)
    system_list = [system for i in range(n)]
    super_system = oqupy.SuperTimeDependentSystemWithField(system_list, field_eom=field_eom)

    start_time = time.perf_counter()
    contractions.compute_dynamics_with_field_multiple_systems(super_system, initial_field=initial_field, 
                                                                                        initial_state_list=initial_state_list, 
                                                                             start_time=start_time,
                                                                                        process_tensor_list = process_tensor
                                                                                        )
    end_time = time.perf_counter()
    time_taken = end_time - start_time

    create_csv(filename, n, time_taken, overwrite = overwrite)

In [3]:
# find runtime for computing dynamics of super-systems containing 1 to 10 systems
for i in range(1,11):
    compute_n_systems(i, "runtime_data.csv")


--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:07
Elapsed time: 7.3s
CSV runtime_data.csv created successfully
--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:13
Elapsed time: 13.9s
CSV runtime_data.csv updated successfully
--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:20
Elapsed time: 20.4s
CSV runtime_data.csv updated successfully
--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:27
Elapsed time: 27.6s
CSV runtime_data.csv updated successfully
--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:32
Elapsed time: 32.7s
CSV runtime_data.csv updated successfully
--> Compute dynamics with field:
100.0%  156 of  156 [########################################] 00:00:37
Elapsed time: 37.5s
CSV runtime_data.csv updated successfully